<a href="https://colab.research.google.com/github/imad10902/Automated-Text-Summarization-Project/blob/main/Automated_Text_Summarization_Project_Imad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Getting dataset

In [ ]:
pd.read_parquet('CNN_data.parquet').to_csv('CNN_data.csv')
df = pd.read_csv('CNN_data.csv')

In [ ]:
df= df.drop(df.columns[0], axis=1)

In [ ]:
df.head()

,article,highlights,id
0,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...,a4942dd663020ca54575471657a0af38d82897d6
1,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend . Leag...,4157bc4da185971e2742f349d69a037343bc0d95
2,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...,60736693e3b1b32d14337a317190c6606e879a85
3,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...,8cdf9cc3ed0276b7a7944cc18ba459355b5984ad
4,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ...",2e6613d531843515bf5401286cc3e45c4df530d2


In [ ]:
df.shape

(13368, 3)

## Cleaning data

In [ ]:
df['article'] = df['article'].str.replace(r'\(CNN\)', '').str.replace(r'\(|\)', '')

<ipython-input-53-159247a65099>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['article'] = df['article'].str.replace(r'\(CNN\)', '').str.replace(r'\(|\)', '')


In [ ]:
df.head()

,article,highlights,id
0,"Share, and your gift will be multiplied. That ...",Zully Broussard decided to give a kidney to a ...,a4942dd663020ca54575471657a0af38d82897d6
1,"On the 6th of April 1996, San Jose Clash and D...",The 20th MLS season begins this weekend . Leag...,4157bc4da185971e2742f349d69a037343bc0d95
2,"French striker Bafetimbi Gomis, who has a hist...",Bafetimbi Gomis collapses within 10 minutes of...,60736693e3b1b32d14337a317190c6606e879a85
3,It was an act of frustration perhaps more comm...,Rory McIlroy throws club into water at WGC Cad...,8cdf9cc3ed0276b7a7944cc18ba459355b5984ad
4,A Pennsylvania community is pulling together t...,"Cayman Naib, 13, hasn't been heard from since ...",2e6613d531843515bf5401286cc3e45c4df530d2


In [ ]:
!pip install transformers

## Splitting the dataset

In [ ]:
train, test = np.split(df.sample(frac=1), [int(0.991*len(df))])

In [ ]:
input= test.copy()
input.head()

,article,highlights,id
9675,"Real estate mogul Donald Trump, a day after an...",'The Donald' is expected to make an earnest ru...,f9f515bfb5d78f82d6cddde034d235ebfd6ee8fd
2226,Wolfsburg and Borussia Monchengladbach both bo...,Wolfsburg look certain for Champions League af...,21e7832f0b54d09ed4869d78d1134d6fe1a03554
4072,David Beckham has lifted the lid on his son Br...,David Beckham was a guest on James Corden's La...,f2b3c18596e1b782a7356235cf7dbf8e2fcb4de7
8093,A Hong Kong man has been found guilty of murde...,A Hong Kong man killed his parents before dism...,6d1ec4ff8c5121458b9f378234f2afe2c49d2074
5403,Brazil captain Thiago Silva has shown his supp...,Thiago Silva launches a scathing attack on Con...,1bb84624e86dff4a754da666647cf4a54a882f16


## Using BART model

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


## Function to summarize text

In [ ]:
def generate_summary(text):
    inputs = tokenizer([text], truncation=True, max_length=1024, return_tensors='pt')

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=70, num_beams=5)

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary

In [ ]:
input['highlights'] = input['article'].apply(generate_summary)

In [ ]:
input.head()

,article,highlights,id
9675,"Real estate mogul Donald Trump, a day after an...",'We need people in Washington that know how to...,f9f515bfb5d78f82d6cddde034d235ebfd6ee8fd
2226,Wolfsburg and Borussia Monchengladbach both bo...,Wolfsburg and Borussia Monchengladbach both bo...,21e7832f0b54d09ed4869d78d1134d6fe1a03554
4072,David Beckham has lifted the lid on his son Br...,David Beckham was a guest on James Corden's La...,f2b3c18596e1b782a7356235cf7dbf8e2fcb4de7
8093,A Hong Kong man has been found guilty of murde...,"Henry Chau Hoi-lueng, 31, has been found guilt...",6d1ec4ff8c5121458b9f378234f2afe2c49d2074
5403,Brazil captain Thiago Silva has shown his supp...,Thiago Silva was recorded on the phone with UF...,1bb84624e86dff4a754da666647cf4a54a882f16


## Comparison between the summaries by BART and already provided data

In [ ]:
input['highlights'].iloc[2]

"David Beckham was a guest on James Corden's Late Late Show in America. He told the story of his son Brooklyn's first date, at a sushi bar. The former Manchester United midfielder watched on from a nearby table in a London restaurant. Brooklyn is looking to follow in his father's footsteps by becoming a professional footballer."

In [ ]:
test['highlights'].iloc[2]

"David Beckham was a guest on James Corden's Late Late Show . He told the story of son Brooklyn's first date, at a London sushi bar . Beckham watched the date from afar under instruction from wife Victoria . He also revealed that 16-year-old Brooklyn has a weekend job at a cafe . WATCH: Beckham strips off with Corden for a spoof underwear advert ."

## Evaluation Metrics

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
score=[]
for input_summary, test_summary in zip(input['highlights'].iloc[20:50], test['highlights'].iloc[20:50]):
  input_text= [input_summary.split()]
  test_text= test_summary.split()
  blue_score = sentence_bleu(input_text, test_text)
  score.append(blue_score)

In [ ]:
mean_score= sum(score)/ len(score)

In [ ]:
mean_score

0.11446491075417614

## Output csv file

In [ ]:
input.to_csv('Output.csv', index=False)